In [2]:
import src.main as lc
import torch, os

# Filepaths
HHD = "/volumes/Ultra Touch"
MLP_LOC = HHD + "/MLP/cut"
COMPRESSED_SAVELOC = HHD + "/MLP/Compressed-test-v2"
DECOMPRESSED_SAVELOC = HHD + "/MLP/Decompressed-test-v2"

# Base dictionary (for decompressor to understand structure of the model)
BASE_DICT = torch.load(MLP_LOC + "/frame_0.ckpt")

# 1. Compression of MLP model set

In [5]:
lc.compress_set_torch(MLP_LOC, COMPRESSED_SAVELOC)

Loading: /volumes/Ultra Touch/MLP/cut/frame_0.ckpt
Delta Compression on: frame_1.ckpt
Loading: /volumes/Ultra Touch/MLP/cut/frame_1.ckpt
Delta Compression on: frame_2.ckpt
Loading: /volumes/Ultra Touch/MLP/cut/frame_2.ckpt
Delta Compression on: frame_3.ckpt
Loading: /volumes/Ultra Touch/MLP/cut/frame_3.ckpt
Delta Compression on: frame_4.ckpt
Loading: /volumes/Ultra Touch/MLP/cut/frame_4.ckpt
Delta Compression on: frame_5.ckpt
Loading: /volumes/Ultra Touch/MLP/cut/frame_5.ckpt
Delta Compression on: frame_6.ckpt
Loading: /volumes/Ultra Touch/MLP/cut/frame_6.ckpt
Delta Compression on: frame_7.ckpt
Loading: /volumes/Ultra Touch/MLP/cut/frame_7.ckpt
Delta Compression on: frame_8.ckpt
Loading: /volumes/Ultra Touch/MLP/cut/frame_8.ckpt
Delta Compression on: frame_9.ckpt
Loading: /volumes/Ultra Touch/MLP/cut/frame_9.ckpt
Saving Compressed Format: frame_0.ckpt
Saving Compressed Format: frame_1.ckpt
Saving Compressed Format: frame_2.ckpt
Saving Compressed Format: frame_3.ckpt
Saving Compressed F

# 2. Decompression from Compressed Set

In [6]:
lc.load_compressed_set(COMPRESSED_SAVELOC, DECOMPRESSED_SAVELOC, BASE_DICT)

Decompressing for: compressed_frame_0.pt
Decompressing for: compressed_frame_1.pt
Decompressing for: compressed_frame_2.pt
Decompressing for: compressed_frame_3.pt
Decompressing for: compressed_frame_4.pt
Decompressing for: compressed_frame_5.pt
Decompressing for: compressed_frame_6.pt
Decompressing for: compressed_frame_7.pt
Decompressing for: compressed_frame_8.pt
Decompressing for: compressed_frame_9.pt
Saving Decompressed Model at: decompressed_frame_0.pt
Saving Decompressed Model at: decompressed_frame_1.pt
Saving Decompressed Model at: decompressed_frame_2.pt
Saving Decompressed Model at: decompressed_frame_3.pt
Saving Decompressed Model at: decompressed_frame_4.pt
Saving Decompressed Model at: decompressed_frame_5.pt
Saving Decompressed Model at: decompressed_frame_6.pt
Saving Decompressed Model at: decompressed_frame_7.pt
Saving Decompressed Model at: decompressed_frame_8.pt
Saving Decompressed Model at: decompressed_frame_9.pt


# 3. Compression Ratio

In [7]:
uncompressed_size = sum(os.path.getsize(MLP_LOC + "/" + f) for f in os.listdir(MLP_LOC))
compressed_size = sum(os.path.getsize(COMPRESSED_SAVELOC + "/" + f) for f in os.listdir(COMPRESSED_SAVELOC))

print("RESULTS")
print("Uncompressed MLP File Size (In MB): " + str(uncompressed_size / (10**6)))
print("Compressed MLP File Size (In MB): " + str(compressed_size / (10**6)))

# Compression Ratio = uncompressed / compressed

print("Compression Ratio (.ckpt VS LC-Checkpoint):")
print("{}%".format(round((uncompressed_size / compressed_size), 3) * 100))

# Space Savings = 1 - (compressed / uncompressed)

print("Space Savings (.ckpt VS LC-Checkpoint):")
print("{}%".format(round(1 - (compressed_size / uncompressed_size), 3) * 100))

RESULTS
Uncompressed MLP File Size (In MB): 21.97774
Compressed MLP File Size (In MB): 5.810592
Compression Ratio (.ckpt VS LC-Checkpoint):
378.2%
Space Savings (.ckpt VS LC-Checkpoint):
73.6%


# 5. Save the torch dicts

In [8]:
for i in range(0, 10):
    compressed = lc.read_decompressed_state_dict(DECOMPRESSED_SAVELOC + "/decompressed_frame_{}.pt".format(i))
    torch.save(compressed, HHD + "/MLP/test_v2/frame_{}.pt".format(i))

In [9]:
for i in range(0, 10):
    print("Test original:")
    print(torch.load(HHD + "/MLP/test_v2/frame_{}.pt".format(i))['_model.field_fine.mlp_base.layers.0.weight']
          [2][20:25])
    print("Test restored:")
    print(torch.load(MLP_LOC + "/frame_{}.ckpt".format(i), 
                     map_location = torch.device('cpu'))['_model.field_fine.mlp_base.layers.0.weight']
                     [2][20:25])

Test original:
tensor([-0.1254,  0.4251, -0.4375, -0.7639,  0.2837])
Test restored:
tensor([-0.1254,  0.4251, -0.4375, -0.7639,  0.2837])
Test original:
tensor([-0.0327,  0.2432, -0.2558, -0.5822,  0.1019])
Test restored:
tensor([-0.0295,  0.2147, -0.2122, -0.6136,  0.0597])
Test original:
tensor([-0.0560,  0.4247, -0.4369, -0.4007,  0.1945])
Test restored:
tensor([-0.0575,  0.3761, -0.3979, -0.4369,  0.1776])
Test original:
tensor([-0.0326,  0.2442, -0.1017, -0.7356,  0.2411])
Test restored:
tensor([-0.0290,  0.1855, -0.1786, -0.7269,  0.2282])
Test original:
tensor([-0.1245,  0.1524, -0.2765, -0.6440,  0.2178])
Test restored:
tensor([-0.0953,  0.1610, -0.3166, -0.6131,  0.2118])
Test original:
tensor([-0.0778,  0.3326, -0.3231, -0.7367,  0.3105])
Test restored:
tensor([-0.0865,  0.3938, -0.3289, -0.7656,  0.3250])
Test original:
tensor([-0.0836,  0.1500, -0.2765, -0.5539,  0.2638])
Test restored:
tensor([-0.0838,  0.2075, -0.2686, -0.5848,  0.2575])
Test original:
tensor([-0.0602,  0